In [4]:
!pip install open_clip_torch==2.23.0 transformers==4.35.2 matplotlib

  Using cached open_clip_torch-2.23.0-py3-none-any.whl.metadata (30 kB)
  Using cached transformers-4.35.2-py3-none-any.whl.metadata (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 923.3 kB/s eta 0:00:00 0:00:01
  Using cached ftfy-6.2.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.22.1-py3-none-any.whl.metadata (12 kB)
  Using cached protobuf-5.26.0-cp37-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached timm-0.9.16-py3-none-any.whl.metadata (38 kB)
  Using cached filelock-3.13.3-py3-none-any.whl.metadata (2.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.meta

In [1]:
from open_clip import create_model_from_pretrained, get_tokenizer # works on open-clip-torch>=2.23.0, timm>=0.9.8

model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

/home/haoyu/anaconda3/envs/torch2.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/haoyu/anaconda3/envs/torch2.2/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/haoyu/anaconda3/envs/torch2.2/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [12]:
import torch
import numpy as np
from typing import Tuple, List

template = 'This is a PET image of '

labels = [
    'synthesized fulldose',
    'original fulldose'
]

maxn = 0.004
scale = 1e4

# low = np.fromfile('/Users/HalveLuve/Downloads/loss_2024_03_21_19_56_43/P-67292_5_3.dat', dtype=np.float32).reshape((256, 256, 256)) * scale
ddim_25 = np.fromfile('/home/haoyu/ddpm-pet-torch/results/predict_out/loss_2024_03_10_23_42_17/Diffusion_Epoch500-GLoss0.0018/ddim_25_P-68813_results.img', dtype=np.float32).reshape((253, 256, 256)).transpose(1, 2, 0)
ddim_50 = np.fromfile('/home/haoyu/ddpm-pet-torch/results/predict_out/loss_2024_03_10_23_42_17/Diffusion_Epoch500-GLoss0.0018/ddim_50_P-68813_results.img', dtype=np.float32).reshape((253, 256, 256)).transpose(1, 2, 0)
ddim_100 = np.fromfile('/home/haoyu/ddpm-pet-torch/results/predict_out/loss_2024_03_10_23_42_17/Diffusion_Epoch500-GLoss0.0018/ddim_100_P-68813_results.img', dtype=np.float32).reshape((253, 256, 256)).transpose(1, 2, 0)
ddpm = np.fromfile('/home/haoyu/ddpm-pet-torch/results/predict_out/loss_2024_03_10_23_42_17/Diffusion_Epoch500-GLoss0.0018/ddpm_P-68813_results.img', dtype=np.float32).reshape((253, 256, 256)).transpose(1, 2, 0)
full = np.fromfile('/media/bld/e644a83d-65c3-4f55-a408-bea0bee7f43e/huaijia/IMG_fulldose_1111/P-68813_5_3.dat', dtype=np.float32).reshape((256, 256, 256)) * scale

def preprocess_input(x: np.ndarray) -> Tuple[np.ndarray, List]:
    invalid_z_list = [k for k in range(x.shape[0]) if np.max(x[k, :, :]) > maxn*scale]
    x_rev = np.delete(x, invalid_z_list, 0)

    x_rev /= maxn*scale
    x_rev -= 0.5
    x_rev /= 0.5

    return x_rev, invalid_z_list

def postprocess_output(x):
    x *= 0.5
    x += 0.5
    x *= maxn*scale
    return x

full, invalid_z_list = preprocess_input(full)
full = postprocess_output(full).transpose(1, 2, 0)


In [4]:
# 22th ~ 222th
from skimage import transform

ddim_25_slice = torch.Tensor(transform.resize(ddim_25[:, :, 125], (224, 224), order=3)).repeat(3, 1, 1)
ddim_50_slice = torch.Tensor(transform.resize(ddim_50[:, :, 125], (224, 224), order=3)).repeat(3, 1, 1)
ddim_100_slice = torch.Tensor(transform.resize(ddim_100[:, :, 125], (224, 224), order=3)).repeat(3, 1, 1)
ddpm_slice = torch.Tensor(transform.resize(ddpm[:, :, 125], (224, 224), order=3)).repeat(3, 1, 1)
full_slice = torch.Tensor(transform.resize(full[:, :, 125], (224, 224), order=3)).repeat(3, 1, 1)

full_slice.shape

torch.Size([3, 224, 224])

In [5]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
model.to(device)
model.eval()

context_len = 256

imgs = torch.stack([ddim_25_slice, ddim_50_slice, ddim_100_slice, ddpm_slice, full_slice]).to(device)
txts = tokenizer([l for l in labels], context_length=context_len).to(device)

with torch.no_grad():
    image_features, text_features, logit_scale = model(imgs, txts)

    logits = (logit_scale * image_features @ text_features.t()).detach().softmax(dim=-1)
    sorted_indices = torch.argsort(logits, dim=-1, descending=True)

    logits = logits.cpu().numpy()
    sorted_indices = sorted_indices.cpu().numpy()

top_k = -1
test_imgs = ['DDIM 25', 'DDIM 50', 'DDIM 100', 'DDPM', 'Fulldose']
for i, img in enumerate(test_imgs):
    pred = labels[sorted_indices[i][0]]

    top_k = len(labels) if top_k == -1 else top_k
    print(img.split('/')[-1] + ':')
    for j in range(top_k):
        jth_index = sorted_indices[i][j]
        print(f'{labels[jth_index]}: {logits[i][jth_index]}')
    print('\n')

DDIM 25:
original fulldose: 0.851511538028717
synthesized fulldose: 0.14848846197128296


DDIM 50:
original fulldose: 0.8843926787376404
synthesized fulldose: 0.11560727655887604


DDIM 100:
original fulldose: 0.8924494385719299
synthesized fulldose: 0.10755056142807007


DDPM:
original fulldose: 0.9130849242210388
synthesized fulldose: 0.08691509068012238


Fulldose:
original fulldose: 0.9409277439117432
synthesized fulldose: 0.05907227843999863




In [13]:
from tqdm import tqdm
from skimage import transform

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
model.to(device)
model.eval()

context_len = 256
txts = tokenizer([l for l in labels], context_length=context_len).to(device)
start = 21
end = 221

logits_list = []

with torch.no_grad():
    for idx in tqdm(range(start, end)): 
        ddim_25_slice = torch.Tensor(transform.resize(ddim_25[:, :, idx], (224, 224), order=3)).repeat(3, 1, 1)
        ddim_50_slice = torch.Tensor(transform.resize(ddim_50[:, :, idx], (224, 224), order=3)).repeat(3, 1, 1)
        ddim_100_slice = torch.Tensor(transform.resize(ddim_100[:, :, idx], (224, 224), order=3)).repeat(3, 1, 1)
        ddpm_slice = torch.Tensor(transform.resize(ddpm[:, :, idx], (224, 224), order=3)).repeat(3, 1, 1)
        full_slice = torch.Tensor(transform.resize(full[:, :, idx], (224, 224), order=3)).repeat(3, 1, 1)
        imgs = torch.stack([ddim_25_slice, ddim_50_slice, ddim_100_slice, ddpm_slice, full_slice]).to(device)
        image_features, text_features, logit_scale = model(imgs, txts)

        logits = (logit_scale * image_features @ text_features.t()).detach().softmax(dim=-1)

        logits_list.append(logits.cpu())

    logits_mean = torch.mean(torch.stack(logits_list), axis=0)
    sorted_indices = torch.argsort(logits_mean, dim=-1, descending=True)
    logits_mean = logits_mean.cpu().numpy()
    sorted_indices = sorted_indices.cpu().numpy()
top_k = -1
test_imgs = ['DDIM 25', 'DDIM 50', 'DDIM 100', 'DDPM', 'Fulldose']
for i, img in enumerate(test_imgs):
    pred = labels[sorted_indices[i][0]]

    top_k = len(labels) if top_k == -1 else top_k
    print(img.split('/')[-1] + ':')
    for j in range(top_k):
        jth_index = sorted_indices[i][j]
        print(f'{labels[jth_index]}: {logits_mean[i][jth_index]}')
    print('\n')

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:31<00:00,  2.19it/s]

DDIM 25:
original fulldose: 0.9411419034004211
synthesized fulldose: 0.05885815620422363


DDIM 50:
original fulldose: 0.9489997029304504
synthesized fulldose: 0.051000241190195084


DDIM 100:
original fulldose: 0.9500135779380798
synthesized fulldose: 0.049986399710178375


DDPM:
original fulldose: 0.971585214138031
synthesized fulldose: 0.02841479517519474


Fulldose:
original fulldose: 0.9859868884086609
synthesized fulldose: 0.0140131376683712


